In [1]:
import numpy as np
import h5py
import keras
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense, Activation, Embedding
from keras.layers import Dropout, TimeDistributed, Bidirectional
from keras.layers import LSTM, ConvLSTM2D, BatchNormalization
from keras import regularizers
from keras import losses
import tensorflow as tf

import time

Using TensorFlow backend.


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/Sundar_Notebooks")
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Archive.zip		    CNN_results_log_Sundar.docx   X_test.npy
 BasicCNN.ipynb		    person_test.npy		  X_train_valid.npy
 BasicCNN-varshinee.ipynb   person_train_valid.npy	  y_test.npy
 BiRNN_Sundar.ipynb	    RNN_Mathanky-66.ipynb	  y_train_valid.npy
'CNN results log.docx'	    RNN_Sundar.ipynb


In [3]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

X_train_valid = np.swapaxes(X_train_valid,1,2)
X_test= np.swapaxes(X_test,1,2)

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

y_train_valid -= 769
y_test -= 769

#person_train_valid -= 769
#person_test -= 769
#y_train_valid = keras.utils.to_categorical(y_train_valid, 4)
#y_test = keras.utils.to_categorical(y_test, 4)

Training/Valid data shape: (2115, 1000, 25)
Test data shape: (443, 1000, 25)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
import pywt
import matplotlib.pyplot as plt

feat_extract = False
num_signals = 25 ### 25 if using EOG, 22 otherwise

if feat_extract:
    mother = 'db4'
    levels = 3
    mode = 'zero'
    feats_train_valid = pywt.wavedec(X_train_valid,mother,axis = 1, level = levels, mode = mode)
    feats_train_valid = np.concatenate(feats_train,axis = 1)
    feats_test = pywt.wavedec(X_test,mother,axis = 1, level = levels, mode = mode)
    feats_test = np.concatenate(feats_test,axis = 1)
else:
    feats_train_valid = X_train_valid
    feats_test = X_test

feats_train_valid = feats_train_valid[:,:,0:num_signals]
feats_test = feats_test[:,:,0:num_signals]

feat_scale = 'on' ####making features 0 mean and unit variance

if feat_scale == 'on':
    temp = feats_train_valid - feats_train_valid.mean(axis=(0,1))
    temp = temp/feats_train_valid.std(axis=(0,1))
    feats_train_valid = temp
    temp = feats_test - feats_test.mean(axis=(0,1))
    temp = temp/feats_test.std(axis=(0,1))
    feats_test = temp
    
# f, ax = plt.subplots(5,5,figsize=(15,10))

# for i in np.arange(22):
#     idx = i%5
#     idy = i//5
#     ax[idy,idx].plot(feats_train[45,:,i])
# plt.show()

exp_mode = 'all' ## choices 'all','one_subject_train','one_subject_test'
val_split = 0.1

if exp_mode == 'one_subject_train':
    subject = 0
    temp = feats_train_valid[person_train_valid[:,0]==subject]
    mask = np.random.permutation(temp.shape[0])
    mask_train = mask[0:int((1-val_split)*mask.shape[0])]
    mask_val = mask[int((1-val_split)*mask.shape[0]):]
    feats_train = temp[mask_train]
    feats_val = temp[mask_val]
    temp2 = y_train_valid[person_train_valid[:,0]==subject]
    y_train = temp2[mask_train]
    y_val = temp2[mask_val]
    
elif exp_mode == 'one_subject_test':
    subject = 0
    temp = feats_train_valid[person_train_valid[:,0]!=subject]
    mask = np.random.permutation(temp.shape[0])
    mask_train = mask[0:int((1-val_split)*mask.shape[0])]
    mask_val = mask[int((1-val_split)*mask.shape[0]):]
    feats_train = temp[mask_train]
    feats_val = temp[mask_val]
    temp2 = y_train_valid[person_train_valid[:,0]!=subject]
    y_train = temp2[mask_train]
    y_val = temp2[mask_val]
    
    feats_test = None
    y_test = None
    feats_test = feats_train_valid[person_train_valid[:,0]==subject]
    y_test = y_train_valid[person_train_valid[:,0]==subject]

elif exp_mode == 'all':
    temp = feats_train_valid
    mask = np.random.permutation(temp.shape[0])
    mask_train = mask[0:int((1-val_split)*mask.shape[0])]
    mask_val = mask[int((1-val_split)*mask.shape[0]):]
    feats_train = temp[mask_train]
    feats_val = temp[mask_val]
    temp2 = y_train_valid
    y_train = temp2[mask_train]
    y_val = temp2[mask_val]
    

In [0]:
def rnn_model(features, labels, mode, params):
  
  num_hidden_layers = len(params['hidden_layers'])  
  
  input_layer = features["x"]    
  
#   outputs = conv1 = tf.layers.conv1d(
#       inputs=input_layer,
#       filters=64,
#       kernel_size=[5],
#       padding="same",
#       activation=tf.nn.relu)
  
  
  
  cell1 = tf.nn.rnn_cell.LSTMCell(name = 'cell1', num_units = params['hidden_layers'][0])
  outputs, state = tf.nn.dynamic_rnn(cell=cell1,
                                   inputs=input_layer,
                                   dtype=tf.float64)
  outputs = tf.nn.relu(outputs)      
  outputs = tf.layers.dropout(inputs=outputs, rate=0.5, training=(mode==tf.estimator.ModeKeys.TRAIN))
  outputs = tf.layers.batch_normalization(inputs=outputs)
  
  for i in range(num_hidden_layers-1):  
      cell = tf.nn.rnn_cell.LSTMCell(name = 'cell'+str(i+2),num_units = params['hidden_layers'][i+1])
      outputs, state = tf.nn.dynamic_rnn(cell=cell,
                                       inputs=outputs,
                                       dtype=tf.float64)
      outputs = tf.nn.relu(outputs)
      outputs = tf.layers.dropout(inputs=outputs, rate=0.5, training=(mode==tf.estimator.ModeKeys.TRAIN))
      outputs = tf.layers.batch_normalization(inputs=outputs)
  
  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs) 
  
#   outputs = tf.layers.dense(inputs=outputs, units=64)
#   outputs = tf.nn.relu(outputs)
  
  outputs = tf.layers.dropout(inputs=outputs, rate=0.1, training=(mode==tf.estimator.ModeKeys.TRAIN))
  outputs = tf.layers.batch_normalization(inputs=outputs)

  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  tf.summary.scalar('loss', loss)
      

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                      sess.graph)
  test_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/test')

In [0]:
import warnings 
warnings.simplefilter('ignore')

In [12]:
start = time.time()
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="model/", 
                                        params = {'hidden_layers' : [128,128], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
#logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": feats_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=10)
   #,hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": feats_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_val_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": feats_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
eval_val_results = eeg_classifier.evaluate(input_fn=eval_val_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": feats_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Validation results are:',eval_val_results)
print('Test results are:',eval_test_results)
end = time.time()
print('Time taken in min:',(end-start)//60)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f630bdb5cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalize